In [2]:
import json
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from modules.faiss_retriever import BookRetriever

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

book_info_path = 'database/book_info.json'

try:
    with open(book_info_path, 'r', encoding='utf-8') as file:
        book_info = json.load(file)
except Exception as e:
    print(f"书籍数据加载失败：{e}")

TPM_limit = 1000000

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

book_retriever = BookRetriever("三体", embeddings, book_info_path)

成功加载书籍: 三体


In [2]:
# 加载小说进Retriever
import tiktoken
novel_path = "novels/TOP/《三体》（实体版1-3全本）作者：刘慈欣.txt"
encoding = tiktoken.encoding_for_model("text-embedding-3-small")
loader = TextLoader(novel_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
)
docs = text_splitter.split_documents(documents)
book_retriever.add_large_documents(docs, encoding, TPM_limit=TPM_limit)
book_retriever.save_local()

RuntimeError: Error loading novels/TOP/《三体》（实体版1-3全本）作者：刘慈欣.txt

In [2]:
book_retriever.search("叶文洁是谁")

[(Document(metadata={'source': 'novels/TOP/《三体》（实体版1-3全本）作者：刘慈欣.txt'}, page_content='“不，那哪行！”白沐霖连连摆手说，“你们建设兵团的女战士，白天干的都是男同志的活儿，快回去休息吧，明天六点就要上山呢。哦，文洁，我后天就要回师部了，我会把你的情况向上级反映一下，也许能帮上忙呢。”\n\u3000\u3000“谢谢，不过我觉得这里很好，挺安静的。”文洁看着月光下大兴安岭朦胧的林海说。\n\u3000\u3000“你是不是在逃避什么？”\n\u3000\u3000“我走了。”叶文洁轻声说，转身离去。\n\u3000\u3000白沐霖看着她那纤细的身影在月光下消失，然后，他抬头遥望文洁刚才看过的林海，看到远方的雷达峰上，巨大的天线又缓缓立起，闪着金属的冷光。\n\u3000\u3000三个星期后的一天中午，叶文洁被从伐木场紧急召回连部。一走进办公室，她就发现气氛不对，连长和指导员都在，还有一个表情冷峻的陌生人，他面前的办公桌上放着一个黑色的公文包，旁边两件东西显然是从公文包中拿出来的，那是一个信封和一本书，信封是拆开的，书就是那本她看过的《寂静的春天》。\n\u3000\u3000这个年代的人对自己的政治处境都有一种特殊的敏感，而这种敏感在叶文洁身上更强烈一些，她顿时感到周围的世界像一个口袋般收紧，一切都向她挤压过来。\n\u3000\u3000“叶文洁，这是师政治部来调查的张主任，”指导员指指陌生人说， “希望你配合，要讲实话。”\n\u3000\u3000“这封信是你写的吗？”张主任问，同时从信封中抽出信来。叶文洁伸手去拿，但张主任没给她，仍把信拿在自己手中，一页一页翻给她看，终于翻到了她想看的最后一页，落款上没有姓名，只写着“革命群众”四个字。\n\u3000\u3000“不，不是我写的。”文洁惊恐地摇摇头。\n\u3000\u3000“可这是你的笔迹。”\n\u3000\u3000“是，可我是帮别人抄的。”\n\u3000\u3000“帮谁？”\n\u3000\u3000平时在连队遇到什么事，叶文洁很少为自己申辩，所有的亏都默默地吃了，所有的委屈都默默地承受，更不用说牵连别人了。但这次不同，她很清楚这意味着什么。\n\u3000\u3000“是帮那位上星期到连队来采访的《大生产报

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4o-mini')
retriever = book_retriever.vector_store.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """根据以下内容对小说进行续写：
{context}

需要续写的开头： {question}
"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("艾AA和云天明被困在黑域之中")

'艾AA和云天明被困在黑域之中，四周是一片无尽的黑暗，连光线也似乎无法穿透。没有方向，没有声音，只有无边的寂静和压迫感。云天明心中升起一丝焦虑，虽然他再一次与程心相会，但此刻却被困在这样一个陌生而无情的空间中。他伸出手，试图在黑暗中找到什么，但手指碰触到的只是虚空。\n\n“我们该怎么办？”艾AA的声音在黑暗中显得格外微弱，似乎连空气都在吞噬她的声音。\n\n“别担心，我们一定能找到出路。”云天明努力让自己保持冷静，心中却暗自焦虑。他知道在这样的黑域中，时间的流逝是无法估量的，任何不慎都可能导致无法逆转的结果。\n\n“程心呢？她在哪里？”艾AA的声音中透出急切。\n\n“我不知道，但我们不能放弃希望。”云天明摇了摇头，心中隐隐感到不安。他们曾经在光明与黑暗之间挣扎过，但这次的黑域似乎比以往任何一次都要深邃和绝望。\n\n“你觉得我们是怎么来到这里的？”艾AA问。\n\n云天明思索着：“我们应该是在——”他的话未能说完，周围的黑暗突然开始扭曲，似乎有某种力量在悄然移动。云天明的心跳加速，他感到一种强烈的不安。\n\n“快！我们得想办法离开这里！”艾AA紧张地说，眼中闪烁着恐惧的光芒。\n\n就在此时，黑暗的尽头似乎出现了一道微弱的光线，起初只是微不足道的闪烁，随后逐渐扩大，仿佛在呼唤着他们。\n\n“那是什么？”艾AA指着光线，眼中闪过一丝希望。\n\n“我们去看看！”云天明果断地说道，心中燃起一丝勇气。他们开始向光线走去，虽然脚下的路依旧模糊，但光的存在让他们感到了一丝温暖。\n\n随着他们不断接近，光线逐渐变得明亮，周围的黑暗似乎也在慢慢褪去。云天明和艾AA的心中充满了期待，仿佛前方有着未知的答案。\n\n但就在他们即将触碰到光线的瞬间，黑暗再次猛然收缩，像是一个巨大的黑色漩涡，将他们卷了进去。云天明拼命挣扎，但无论如何也无法逃脱。\n\n“云天明！”艾AA的声音在耳边回荡，直到最后一刻，云天明仍然能感觉到艾AA的手紧紧握住他的。\n\n在黑暗与光明的交替中，云天明的意识渐渐模糊，但他心中始终铭记着程心的影子。无论在多么绝望的境地，他们的心灵永远相连。\n\n当他再次睁开眼睛时，周围的环境已然不同。他们不再身处黑域，而是一个充满生机的世界——蓝色的草地、淡黄色的天空，还有那红色的太阳。\n\n“我们……回来了？”艾AA的声音中满是震惊。\n\n“看来是的。”云天